In [50]:
#import relevent libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
print("Libraries imported successfully")

Libraries imported successfully


In [51]:
# load train and test datasets
train=pd.read_csv('datasets/train.csv')
train.head(10)


      

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [52]:
input=np.asfarray(train.iloc[:,1:])

output = np.zeros((42000, 10))
for i in range(42000):
    output[i, np.array(train.label)[i]] = 0.99
output = output.astype(float)
output=np.transpose(output)
input=np.transpose(input)

m=4200
input_nodes =input.shape[0]
num_layer = 3 # number of layers
h1 = 200       # hidden layer 1
h2 = 150        # hidden layer 2
output_nodes = 10
learning_rate=0.01

In [53]:
print("The size of input is {}.".format(input_nodes))
print("The size of output is {}.".format(output_nodes))
print(input.shape)
#print(input)
print("output")
print(output.shape)
print(output)

The size of input is 784.
The size of output is 10.
(784, 42000)
output
(10, 42000)
[[0.   0.99 0.   ... 0.   0.   0.  ]
 [0.99 0.   0.99 ... 0.   0.   0.  ]
 [0.   0.   0.   ... 0.   0.   0.  ]
 ...
 [0.   0.   0.   ... 0.99 0.   0.  ]
 [0.   0.   0.   ... 0.   0.   0.  ]
 [0.   0.   0.   ... 0.   0.   0.99]]


In [54]:
#define sigmoid function
def sigmoid(z):
    g=1/(1+np.exp(-z))
    
    return g

In [55]:
#initialize the required parameters with array of appropriate dimensions
def parameters_initialization(input_nodes, h1, h2, output_nodes):
    
    #weights are initialized in each layer with random values
    W1=np.random.randn(h1, input_nodes)
    W2=np.random.randn(h2, h1)
    W3=np.random.randn(output_nodes, h2)
    b1=np.zeros((h1,1))
    b2=np.zeros((h2,1))
    b3=np.zeros((output_nodes,1))
    
    return W1,W2,W3,b1,b2,b3
    

In [56]:
#
def forward_propagation(input, W1, W2, W3,b1,b2,b3):
    
    #calculate linear equation of input times weight
    
    Z1=np.add(np.dot(W1,input),b1)
    A1=sigmoid(Z1)
    Z2=np.add(np.dot(W2,A1),b2)
    A2=sigmoid(Z2)
    Z3=np.add(np.dot(W3,A2),b3)
    A3=sigmoid(Z3)
    
    
    
#     print("W1")
#     print(W1.shape)
#     print("A2")
#     print(A2.shape)
    
    return A3,A2,A1
    

In [57]:
#calculate cost function
def cost_function(output,A3):
    cost=cost=(-1/m)*np.sum((output*np.log(A3))+(1-output*np.log(1-A3)))
    
    return cost


In [58]:
def back_propagation(input,output,learning_rate,A3,A2,A1,W3,W2,W1,b1,b2,b3):
#     print("A1")
#     print(A1.shape)
    dZ3=A3-output
    dZ2=np.dot(W3.T,dZ3)*(1-np.power(A2,2))
    dZ1=np.dot(W2.T,dZ2)*(1-np.power(A1,2))
#     print("W1")
#     print(W1.shape)
#     print("A1")
#     print(A1.shape)
#     print("DZ2")
#     print(dZ2.shape)

    dW3=(1/m)*np.dot(dZ3,A2.T)
    dW2=(1/m)*np.dot(dZ2,A1.T)
    dW1=(1/m)*np.dot(dZ1,input.T)
    
    db3=(1/m)*np.sum(dZ3, axis=1, keepdims=True)
    db2=(1/m)*np.sum(dZ2, axis=1, keepdims=True)
    db1=(1/m)*np.sum(dZ1, axis=1, keepdims=True)
    
    
    W1=W1-learning_rate*dW1
    W2=W2-learning_rate*dW2
    W3=W3-learning_rate*dW3
    
    b1=b1-learning_rate*db1
    b2=b2-learning_rate*db2
    b3=b3-learning_rate*db3

    return W1,W2,W3,b1,b2,b3
    
    

In [61]:

epochs=25
W1, W2, W3, b1, b2, b3=parameters_initialization(input_nodes, h1, h2, output_nodes)
for epoch in range(epochs):
    print("Epoch: "+ str(epoch+1)+"/"+str(epochs))   
#     print(W1.shape)
#     print(W2.shape)
#     print(W3.shape)
#     print(input.shape)
#     print(b1.shape)
#     print(b2.shape)
#     print(b3.shape)
    A3,A2,A1=forward_propagation(input, W1, W2, W3,b1,b2,b3)
    cost=cost_function(output,A3)
    print(cost)
    W1,W2,W3,b1,b2,b3=back_propagation(input,output,learning_rate,A3,A2,A1,W3,W2,W1,b1,b2,b3)
#     print("A3")
#     print(A3)
    
    

Epoch: 1/25


C:\Users\Rixant\anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until


-116.69258303718821
Epoch: 2/25
6.879389698218919
Epoch: 3/25
-8.155462109751452
Epoch: 4/25
-39.5005768337038
Epoch: 5/25
-24.46428156009361
Epoch: 6/25
-18.51998226175893
Epoch: 7/25
-24.440968302424633
Epoch: 8/25
-44.05627396009325
Epoch: 9/25
-37.77101916324656
Epoch: 10/25
-52.780789669625015
Epoch: 11/25
-56.848853670147676
Epoch: 12/25
-64.42951343504767
Epoch: 13/25
-69.06384636300685
Epoch: 14/25
-77.91823179214812
Epoch: 15/25
-74.83339840657592
Epoch: 16/25
-78.63046060177444
Epoch: 17/25
-78.35676783381368
Epoch: 18/25
-75.72033007271844
Epoch: 19/25
-75.15801127814466
Epoch: 20/25
-78.48015598459831
Epoch: 21/25
-81.11200821407455
Epoch: 22/25
-82.41482726878219
Epoch: 23/25
-83.07998559132903
Epoch: 24/25
-80.57541697542518
Epoch: 25/25
-85.09857589056074


In [60]:
# input=np.asfarray(np.transpose(train.drop('label',axis=1)))
# epochs=25
# output=np.array(output)
# print(output.shape)
# output=output.T
# print(output.shape)
# print(input.shape)